In [1]:
import ee
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import geemap.foliumap as geemap
import sys

sys.path.append('../../')

from src.ee_scripts.ee_utils import *

ee.Authenticate()
ee.Initialize()

In [2]:
GAUL1 = ee.FeatureCollection('FAO/GAUL/2015/level1')
GAUL0 = ee.FeatureCollection('FAO/GAUL/2015/level0')

germany = GAUL0.filter(ee.Filter.eq('ADM0_NAME', 'Germany'))
provinces = GAUL1.filter(ee.Filter.eq('ADM0_NAME', 'Germany'))
provinces = provinces.toList(provinces.size().getInfo())
subset1_geom = ee.FeatureCollection(provinces.slice(0, int(0.5 * provinces.size().getInfo())))
subset2_geom = ee.FeatureCollection(provinces.slice(int(0.5 * provinces.size().getInfo()), provinces.size().getInfo()))



In [3]:
Map = geemap.Map()
Map.addLayer(subset1_geom, {'color':'red'}, 'subset1')
Map.addLayer(subset2_geom, {'color':'blue'}, 'subset2')
Map.centerObject(germany)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [13]:
roi = subset1_geom
start_date = '2018-01-01'
end_date = '2019-01-01'
year = '2018'
step = 10 #Days to step for averages
export = 10

lucas_subset =  (
    ee.FeatureCollection('projects/ee-ayang115t/assets/lucas_2018_filtered_polygons')
    .filterBounds(roi)
)
s1 = generate_s1_averages(start_date, end_date, roi, step)
histar = extract_histar(roi, start_date, end_date, step)

fused = s1.addBands(histar)

In [8]:
fused.bandNames()


In [14]:
imageVisParam = {
    "opacity": 1,
    "bands": [
        "13_B3_mean_post_20180511",
        "13_B2_mean_post_20180511",
        "13_B1_mean_post_20180511"
    ],
    'min': 0,
    'max': 2500,
    'gamma': 1
}

s1VisParams = {
    
}

Map.addLayer(fused.select("13_VV_20180511").clip(roi), {"min": -25, "max": 0}, "VV")
Map.addLayer(fused.select("13_VH_20180511").clip(roi), {"min": -25, "max": 0}, "VH")
Map.addLayer(fused.clip(roi), imageVisParam, "visual")
Map.addLayer(lucas_subset, {"color":"red"}, "Lucas points")

In [15]:
Map

Map(bottom=3946.0, center=[19.394067895396613, 18.874511718750004], controls=(WidgetControl(options=['position…